In [1]:
# main libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.nn import functional as F
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms, datasets

In [2]:
from torch.utils.tensorboard import SummaryWriter

In [3]:
# custom libraries
root_main = os.getcwd()
os.chdir("..")
import TorchCommon as TC
os.chdir(root_main)

In [4]:
# main pyperparametrs
valid_size=0.2
nrm_mean=0.5
nrm_std=0.5
num_workers=0
root_ds = "D:\GitHub\pytorch-lab\Dataset" # dataset root
root_bm = "D:\GitHub\pytorch-lab\Best_Models" # best models root

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# transform
trans=transforms.Compose([
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((nrm_mean,) , (nrm_std,))
])

In [7]:
#load dataset
train_data=datasets.MNIST(root=root_ds,
                          train=True, transform=trans, download=True)

In [8]:
#sampler
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)

In [9]:
# Network
class Net(nn.Module):
    # Assign Intrinsic Properties of Your Neural Network
    def __init__(self):
        super().__init__()
        # Weights of Layer 1th and Layer 2th Are Intrinsic Properties
        self.fc1 = nn.Linear(784, 256, bias=True)
        self.fc2 = nn.Linear(256, 100, bias=True)
        self.fc3 = nn.Linear(100, 10, bias=True)

    # Wiring of Your Network
    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = F.relu_(x)
        x = self.fc2(x)
        x = F.relu_(x)
        x = self.fc3(x)
        return x

# TensorBoard

### Loss/Accuracy Plot (independently)

In [ ]:
# Hyperparaameter
epochs=5
batch_size=64
lr=0.1

# TrainLoader
train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)

# Loss and Optimizer
model=Net().to(device)
criterion =nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=lr)

# TesnorBoard File
root_tb=f'runs/batch {batch_size} LR {lr}'
writer = SummaryWriter(root_tb)

# Train
step=0
for epoch in range(1,epochs+1):
    for iter_train, (image, label) in enumerate(train_loader, 1):
        # preprocess and feedforward process
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        y_hat = model(image)
        # we want to track loss/acc
        loss = criterion(y_hat, label)
        acc = np.count_nonzero(label.to('cpu').numpy() == y_hat.argmax(dim=1).to('cpu').numpy())
        # updating process
        loss.backward()
        optimizer.step()

        # -NEW- TensorBoard process
        writer.add_scalar("Loss Train", scalar_value= loss, global_step=step)
        writer.add_scalar("Accuracy Train", scalar_value= acc, global_step=step)
        step+=1
        # -NEW- TensorBoard process
writer.close()

### Loss/Accuracy Plot (hold on)

In [37]:
# Hyperparaameter
epochs=5
batch_size=64
lr=0.1

# TrainLoader
train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)

# Loss and Optimizer
model=Net().to(device)
criterion =nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=lr)

# TesnorBoard File
root_tb=f'runs/batch {batch_size} LR {lr}'
writer = SummaryWriter(root_tb)

# Train
step=0
for epoch in range(1,epochs+1):
    for iter_train, (image, label) in enumerate(train_loader, 1):
        # preprocess and feedforward process
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        y_hat = model(image)
        # we want to track loss/acc
        loss = criterion(y_hat, label)
        acc = np.count_nonzero(label.to('cpu').numpy() == y_hat.argmax(dim=1).to('cpu').numpy())
        # updating process
        loss.backward()
        optimizer.step()

        # -NEW- TensorBoard process
        writer.add_scalars("Loss/Accuracy", {'Loss' : loss, 'Accuracy': acc}, global_step=step)
        step+=1
        # -NEW- TensorBoard process
writer.close()

### Histogram (Parameters)

In [50]:
# Hyperparaameter
epochs=5
batch_size=64
lr=0.1

# TrainLoader
train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)

# Loss and Optimizer
model=Net().to(device)
criterion =nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=lr)

# TesnorBoard File
root_tb=f'runs/batch {batch_size} LR {lr}'
writer = SummaryWriter(root_tb)

# Train
step=0
for epoch in range(1,epochs+1):
    for iter_train, (image, label) in enumerate(train_loader, 1):
        # preprocess and feedforward process
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        y_hat = model(image)
        # we want to track loss/acc
        loss = criterion(y_hat, label)
        acc = np.count_nonzero(label.to('cpu').numpy() == y_hat.argmax(dim=1).to('cpu').numpy())
        # updating process
        loss.backward()
        optimizer.step()

        # -NEW- TensorBoard process
        writer.add_histogram("FC1_Weights", model.fc1.weight, global_step=step)
        writer.add_histogram("FC2_Weights", model.fc2.weight, global_step=step)
        writer.add_histogram("FC3_Weights", model.fc3.weight, global_step=step)
        step+=1
        # -NEW- TensorBoard process
writer.close()

### Computation Graph

In [48]:
# Hyperparaameter
batch_size=64

# TrainLoader
train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)

# Loss and Optimizer
model=Net()

# TesnorBoard File
root_tb=f'runs/batch {batch_size}'
writer = SummaryWriter(root_tb)

# -NEW- TensorBoard process
image,label=next(iter(train_loader))
writer.add_graph(model, image)
# -NEW- TensorBoard process
writer.close()

### mini-batch images (every iteration)

In [65]:
# Hyperparaameter
epochs=1
batch_size=61
lr=0.1

# TrainLoader
train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)

# Loss and Optimizer
model=Net().to(device)
criterion =nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=lr)

# TesnorBoard File
root_tb=f'runs/batch {batch_size} LR {lr}'
writer = SummaryWriter(root_tb)

# Train
step=0
for epoch in range(1,epochs+1):
    for iter_train, (image, label) in enumerate(train_loader, 1):
        # preprocess and feedforward process
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        y_hat = model(image)
        # we want to track loss/acc
        loss = criterion(y_hat, label)
        acc = np.count_nonzero(label.to('cpu').numpy() == y_hat.argmax(dim=1).to('cpu').numpy())
        # updating process
        loss.backward()
        optimizer.step()

        # -NEW- TensorBoard process
        writer.add_images("mini=batch data", image, global_step=iter_train)
        step+=1
        # -NEW- TensorBoard process
writer.close()

### mini-batch signals (every iteration)

### Hyperparameters Searching

In [10]:
# Hyperparaameter
epochs=1
for batch_size in [10,100,1000]:
    for lr in [0.1, 0.01, 0.001]:
        # TrainLoader
        train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)

        # Loss and Optimizer
        model=Net().to(device)
        criterion =nn.CrossEntropyLoss()
        optimizer=optim.SGD(model.parameters(), lr=lr)

        # TesnorBoard File
        root_tb=f'runs/batch {batch_size} LR {lr}'
        writer = SummaryWriter(root_tb)

        # Train
        for epoch in range(1,epochs+1):
            loss_train, acc_train = TC.train(model,train_loader, device, optimizer, criterion, epoch)

        # -NEW- TensorBoard process
        writer.add_hparams({'batch_size' : batch_size, 'LR' : lr} ,
                           {'Loss train' : loss_train, 'Accuracy train' : acc_train})
        # -NEW- TensorBoard process
        writer.close()

############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 0.40	Acc_Train : 0.87
############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 0.49	Acc_Train : 0.86
############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 1.50	Acc_Train : 0.60
############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 0.59	Acc_Train : 0.81
############################################################
###### Epoch 1 #############################################
##########

### Embedding Projector

In [20]:
# desired datas that you want visualize
images, labels = train_data.data[:1000], train_data.targets[:1000]
# make TB file
writer = SummaryWriter(f"runs")

In [21]:
# create "image labels" for classes.
label_image  = torch.unsqueeze(images,1)

In [22]:
# create "name labels" for class.
label_name=['0','1','2','3','4','5','6','7','8','9']
# replace every 0,1 label with related string
label_name = [label_name[lab] for lab in labels]

In [23]:
# Features (Dimensions)
features = torch.flatten(images, start_dim=1).to(torch.float32)

In [24]:
writer.add_embedding(features,
                    metadata=label_name,
                     label_img =label_image )
writer.close()